In [23]:
import os
import cv2
import numpy as np
from skimage.feature import hog

def extract_features_and_labels(folder_path):
    features = []
    labels = []

    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                image_path = os.path.join(label_path, filename)
                image = cv2.imread(image_path)
                if image is not None:
                    hog_features = hog_descriptor(image)
                    features.append(hog_features)
                    labels.append(label)

    return np.array(features), np.array(labels)

def hog_descriptor(image):
    if image.ndim > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)


folder_path = 'D:/STUDY/PROJECT/HinhAnh/'
features, labels = extract_features_and_labels(folder_path)

# Lưu các đặc trưng và nhãn vào các tập tin npy
np.save('hog_features.npy', features)
np.save('labels.npy', labels)


C:\Users\Chikuka\AppData\Local\Temp\ipykernel_10368\4039905107.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(features), np.array(labels)


In [9]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from skimage.feature import hog
from skimage import exposure

def extract_hog_features(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply HOG descriptor
    fd, hog_image = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    # Rescale HOG features
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return fd

def extract_features_and_labels(folder_path):
    features = []
    labels = []
    label_mapping = {}  # To store a mapping between folder names and labels
    label_counter = 0

    for label_folder in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label_folder)
        if os.path.isdir(label_path):
            # Try to convert the folder name to an integer label
            try:
                label = int(label_folder)
            except ValueError:
                # If conversion fails, assign a unique label
                label = label_counter
                label_counter += 1
            label_mapping[label] = label_folder

            for image_name in os.listdir(label_path):
                image_path = os.path.join(label_path, image_name)
                image = cv2.imread(image_path)
                if image is not None:
                    # Resize the image to a consistent size
                    resized_image = cv2.resize(image, (128, 128))  # Adjust the size as needed
                    hog_features = extract_hog_features(resized_image)
                    features.append(hog_features)
                    labels.append(label)

    return np.array(features), np.array(labels), label_mapping

# Example usage:
folder_path = 'D:/STUDY/PROJECT/HinhAnh/'
X, y, label_mapping = extract_features_and_labels(folder_path)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Evaluate the model
accuracy = svm_model.score(X_test, y_test)
print("Accuracy:", accuracy)

# Predict labels for test data
y_pred = svm_model.predict(X_test)

# Optionally, you can use the label mapping to interpret the predicted labels
# predicted_labels = [label_mapping[label] for label in y_pred]
# print("Predicted labels:", predicted_labels)


Accuracy: 0.543859649122807
